In [1]:
import datetime
import pandas as pd
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# Search for conjunctions

The cornerstone aspect of the AuroraX search engine is the ability to automatically find conjunctions between ground-based instruments and spacecrafts. One aspect that makes AuroraX's conjunction search different from other solution is that it will only find times where there is ground-based data available. It can also limit results to conjunctions where certain spacecraft instruments were operating. This integration with data availability saves us all valuable minutes and hours when surveying or searching for data that could be helpful for your next paper. 

A great way to get introduced to the AuroraX Conjunction Search capabilities is to head on over the [web page](https://aurorax.space/conjunctionSearch/standard) and play around with UI there. Nearly any request you can do in the web interface, you can also do using the below function.

In [3]:
# search for conjunctions between any THEMIS ASI and any Swarm spacecraft
#
# define timeframe and distance
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 1, 6, 59, 59)
distance = 500

# create ground criteria block
ground = aurorax.search.GroundCriteriaBlock(programs=["themis-asi"])

# create space criteria block
space = aurorax.search.SpaceCriteriaBlock(programs=["swarm"])

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=[ground], space=[space], verbose=True)

[2025-01-24 19:38:02.727387] Search object created
[2025-01-24 19:38:02.747385] Request submitted
[2025-01-24 19:38:02.747456] Request ID: 0f1e1e77-319d-4833-8454-c7c61d632976
[2025-01-24 19:38:02.747484] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/0f1e1e77-319d-4833-8454-c7c61d632976
[2025-01-24 19:38:02.747507] Waiting for data ...
[2025-01-24 19:38:04.161411] Checking for data ...
[2025-01-24 19:38:04.583136] Data is now available
[2025-01-24 19:38:04.583345] Retrieving data ...
[2025-01-24 19:38:05.077522] Retrieved 376.7 kB of data containing 49 records


In [4]:
# output data
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)

# let's show the first 10 conjunctions, sorted by 'start' time
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
43,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
15,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
11,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",282.877438,285.744881,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
38,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",195.554240,298.419001,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
30,nbtrace,2020-01-01T00:11:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",53.391843,441.814398,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:11:00
4,nbtrace,2020-01-01T00:12:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",89.436436,332.998863,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:13:00
17,nbtrace,2020-01-01T00:13:00,2020-01-01T00:13:00,"[DataSource(identifier=54, program='themis-asi...",401.883399,401.883399,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:13:00,2020-01-01T00:13:00
9,nbtrace,2020-01-01T00:13:00,2020-01-01T00:15:00,"[DataSource(identifier=50, program='themis-asi...",105.345328,488.349867,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:15:00
37,nbtrace,2020-01-01T00:13:00,2020-01-01T00:14:00,"[DataSource(identifier=50, program='themis-asi...",215.337413,330.343161,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:13:00
7,nbtrace,2020-01-01T00:44:00,2020-01-01T00:46:00,"[DataSource(identifier=48, program='themis-asi...",114.890751,488.203200,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:45:00,2020-01-01T00:46:00


# Search for conjunctions with metadata filters

Using the metadata filters to help search for conjunctions is one of the more advanced, and highly powerful, tools available. There are many metadata filters for spacecrafts, and some very useful ML-derived filters for ground-based all-sky imagers. 

Let's have a look at a simple example here, where we search for conjunctions filtering for when spacecrafts are in the north polar cap. The regions available to choose from are all directly pulled from SSCWeb. Almost all data used by AuroraX for spacecrafts is from SSCWeb without any alterations, only organization in the AuroraX database to enable the search engine to function.

If you're interested in interacting with the ML-derived metadata for ground-based instruments, have a look at [this crib sheet](https://github.com/aurorax-space/pyaurorax/tree/main/examples/notebooks/search/search_ephemeris_ml.ipynb).

In [5]:
# search for conjunctions between any THEMIS-ASI or REGO instrument, and any Swarm
# instrument where the north B-trace region is 'north polar cap'.
#
# NOTE: this region metadata is not derived by AuroraX, but instead directly from
# SSCWeb. See the above text for more info.
#
# set up search paramters
start = datetime.datetime(2019, 2, 1, 0, 0, 0)
end = datetime.datetime(2019, 2, 10, 23, 59, 59)
ground = [{"programs": ["themis-asi", "rego"]}]
space = [{
    "programs": ["swarm"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [{
            "key": "nbtrace_region",
            "operator": "in",
            "values": ["north polar cap"]
        }]
    }
}]

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=ground, space=space, verbose=True)

[2025-01-24 19:38:05.143239] Search object created
[2025-01-24 19:38:05.168199] Request submitted
[2025-01-24 19:38:05.168267] Request ID: 01e42c9f-57e3-44aa-9363-e054b7bb3cd2
[2025-01-24 19:38:05.168294] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/01e42c9f-57e3-44aa-9363-e054b7bb3cd2
[2025-01-24 19:38:05.168317] Waiting for data ...
[2025-01-24 19:38:06.595848] Checking for data ...
[2025-01-24 19:38:07.018113] Data is now available
[2025-01-24 19:38:07.018342] Retrieving data ...
[2025-01-24 19:38:07.666995] Retrieved 2.1 MB of data containing 273 records


In [6]:
# view the first 10 conjunctions
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
97,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=53, program='themis-asi...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
197,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=80, program='rego', pla...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
229,nbtrace,2019-02-01T02:21:00,2019-02-01T02:22:00,"[DataSource(identifier=83, program='rego', pla...",267.685472,300.984658,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:22:00,2019-02-01T02:21:00
170,nbtrace,2019-02-01T03:55:00,2019-02-01T03:56:00,"[DataSource(identifier=78, program='rego', pla...",471.096543,490.131048,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:56:00,2019-02-01T03:55:00
232,nbtrace,2019-02-01T03:57:00,2019-02-01T03:57:00,"[DataSource(identifier=83, program='rego', pla...",496.774661,496.774661,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:57:00,2019-02-01T03:57:00
151,nbtrace,2019-02-01T04:55:00,2019-02-01T04:56:00,"[DataSource(identifier=83, program='rego', pla...",424.997148,445.453544,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:56:00
252,nbtrace,2019-02-01T04:55:00,2019-02-01T04:55:00,"[DataSource(identifier=83, program='rego', pla...",438.433977,438.433977,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:55:00
174,nbtrace,2019-02-01T05:31:00,2019-02-01T05:31:00,"[DataSource(identifier=78, program='rego', pla...",440.820767,440.820767,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T05:31:00,2019-02-01T05:31:00
262,nbtrace,2019-02-01T06:28:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",234.199376,484.697629,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:28:00
155,nbtrace,2019-02-01T06:29:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",280.074450,466.631937,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:30:00
